In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

#import missingno as msno



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Format Options, display only two decimals on float values
pd.set_option('display.float_format', '{:.2f}'.format)
#pd.options.display.float_format = '{:,}'.format
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
#Import pertinent datasets

co2 = pd.read_csv("/kaggle/input/co2-ghg-emissionsdata/co2_emission.csv")

freedom = pd.read_csv("/kaggle/input/the-economic-freedom-index/economic_freedom_index2019_data.csv", encoding="cp1252")

countries = pd.read_csv("/kaggle/input/countries-of-the-world/countries of the world.csv")

#freedom_world = pd.read_csv("/kaggle/input/cusersmarildownloadsallcsv/all.csv")

In [ ]:
#Display column headings, number of values (not-null), and data type

co2.info()

In [ ]:
freedom.info()

In [ ]:
countries.info()

In [ ]:
freedom_world.info()

In [ ]:
#Replace the , for a . to convert these string type values to float values.

countries['Pop. Density (per sq. mi.)'] = countries['Pop. Density (per sq. mi.)'].str.replace(',','.')
countries['Coastline (coast/area ratio)'] = countries['Coastline (coast/area ratio)'].str.replace(',','.')
countries['Net migration'] = countries['Net migration'].str.replace(',','.')
countries['Infant mortality (per 1000 births)'] = countries['Infant mortality (per 1000 births)'].str.replace(',','.')
countries['Literacy (%)'] = countries['Literacy (%)'].str.replace(',','.')
countries['Phones (per 1000)'] = countries['Phones (per 1000)'].str.replace(',','.')
countries['Arable (%)'] = countries['Arable (%)'].str.replace(',','.')
countries['Crops (%)'] = countries['Crops (%)'].str.replace(',','.')
countries['Other (%)'] = countries['Other (%)'].str.replace(',','.')
countries['Birthrate'] = countries['Birthrate'].str.replace(',','.')
countries['Deathrate'] = countries['Deathrate'].str.replace(',','.')
countries['Agriculture'] = countries['Agriculture'].str.replace(',','.')
countries['Industry'] = countries['Industry'].str.replace(',','.')
countries['Service'] = countries['Service'].str.replace(',','.')

In [ ]:
#Change the data types of these columns to float type using a dictionary

convert_dict = {'Pop. Density (per sq. mi.)': float,
                'Coastline (coast/area ratio)': float,
                'Net migration': float,
                'Infant mortality (per 1000 births)': float,
                'Literacy (%)': float,
                'Phones (per 1000)': float,
                'Arable (%)' : float,
                'Crops (%)': float,
                'Other (%)': float, 
                'Birthrate': float,
                'Deathrate' : float,
                'Agriculture': float,
                'Industry': float,
                'Service' : float
               }
  
countries = countries.astype(convert_dict)
print(countries.dtypes)

In [ ]:
#print firts 5 rows to check the result of our changes above

countries.head()

In [ ]:
#Rename the column heading containing the name of the country, this will help with referential integrity when doing our merge.

co2.rename(columns = {"Entity": "Country_Name", "Annual CO₂ emissions (tonnes )": "CO2_tonnes"}, inplace = True)
freedom.rename(columns = {"Country Name": "Country_Name"}, inplace = True)
countries.rename(columns = {"Country": "Country_Name"}, inplace = True)

co2["Year"] = pd.to_datetime(co2.Year, format='%Y')
countries['Country_Name'] = countries['Country_Name'].str.strip()

In [ ]:
#Filter the column year to only have data between 1960 and 2017

co2_filtered = co2.loc[(co2['Year'] >= '1960-01-01')
                     & (co2['Year'] < '2017-01-01')]

In [ ]:
# For each country name, average values in CO2_tonnes

co2_grp = co2_filtered.groupby('Country_Name')['CO2_tonnes'].mean()

In [ ]:
co2_grp.head()

In [ ]:
co2_index_one = pd.merge(co2_grp, freedom, how="left", on="Country_Name")
co2_index = pd.merge(co2_index_one, countries,how="left", on="Country_Name")



In [ ]:
co2_index.info()

In [ ]:
# counting unique values 
n = co2_index.nunique(axis=0) 
  
print("No.of.unique values in each column :\n", 
      n)

In [ ]:
#Count number of empty rows in each column
co2_index.isnull().sum()

In [ ]:
#Under construction

#msno.matrix(co2_index, fontsize=16)
#plt.show()



In [ ]:
co2_index = co2_index[co2_index['World Rank'].notna()]

In [ ]:
# Top 20 largest polluters by average since the 1960
top_co2 = co2_index.nlargest(20,['CO2_tonnes'])
print(top_co2)

In [ ]:
%%capture
! pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [ ]:
# Correlation table, the closer to 1 or -1, the more impact does the attribute has on our target variable.
co2_index.corr()

In [ ]:
# The most correlated factors on CO2 emmisions

co2_index[['CO2_tonnes','Judical Effectiveness','Population','Area (sq. mi.)','GDP ($ per capita)','Phones (per 1000)']].corr()

In [ ]:
sns.regplot(x="Area (sq. mi.)", y="CO2_tonnes", data=top_co2)
plt.ylim(0,)

In [ ]:
sns.regplot(x="Population", y="CO2_tonnes", data=top_co2)
plt.ylim(0,)

In [ ]:
sns.boxplot(x="Region_x", y="CO2_tonnes", data=top_co2)

In [ ]:
sns.boxplot(x="Climate", y="CO2_tonnes", data=top_co2)

In [ ]:
co2_index.describe(include='all')

In [ ]:
def swarmplot(
    *,
    x=None, y=None,
    hue=None, data=None,
    order=None, hue_order=None,
    dodge=False, orient=None, color=None, palette=None,
    size=5, edgecolor="gray", linewidth=0, ax=None,
    hue_norm=None, fixed_scale=True, formatter=None, warn_thresh=.05,
    **kwargs
):

    p = _CategoricalPlotterNew(
        data=data,
        variables=_CategoricalPlotterNew.get_semantics(locals()),
        order=order,
        orient=orient,
        require_numeric=False,
    )

    if ax is None:
        ax = plt.gca()

    if fixed_scale or p.var_types[p.cat_axis] == "categorical":
        p.scale_categorical(p.cat_axis, order=order, formatter=formatter)

    p._attach(ax)

    if not p.has_xy_data:
        return ax

    palette, hue_order = p._hue_backcompat(color, palette, hue_order)

    color = _default_color(ax.scatter, hue, color, kwargs)

    p.map_hue(palette=palette, order=hue_order, norm=hue_norm)

    # XXX Copying possibly bad default decisions from original code for now
    kwargs.setdefault("zorder", 3)
    size = kwargs.get("s", size)

    if linewidth is None:
        linewidth = size / 10

    kwargs.update(dict(
        s=size ** 2,
        linewidth=linewidth,
    ))

    p.plot_swarms(
        dodge=dodge,
        color=color,
        edgecolor=edgecolor,
        warn_thresh=warn_thresh,
        plot_kws=kwargs,
    )

    # XXX this happens inside a plotting method in the distribution plots
    # but maybe it's better out here? Alternatively, we have an open issue
    # suggesting that _attach could add default axes labels, which seems smart.
    p._add_axis_labels(ax)
    p._adjust_cat_axis(ax, axis=p.cat_axis)

    return ax

In [ ]:
def set_theme(context="notebook", style="darkgrid", palette="deep",
              font="sans-serif", font_scale=1, color_codes=True, rc=None):
    """
    Set aspects of the visual theme for all matplotlib and seaborn plots.
    This function changes the global defaults for all plots using the
    :ref:`matplotlib rcParams system <matplotlib:matplotlib-rcparams>`.
    The themeing is decomposed into several distinct sets of parameter values.
    The options are illustrated in the :doc:`aesthetics <tutorial/aesthetics>`
    and :doc:`color palette <tutorial/color_palettes>` tutorials.
    Parameters
    ----------
    context : string or dict
        Scaling parameters, see :func:`plotting_context`.
    style : string or dict
        Axes style parameters, see :func:`axes_style`.
    palette : string or sequence
        Color palette, see :func:`color_palette`.
    font : string
        Font family, see matplotlib font manager.
    font_scale : float, optional
        Separate scaling factor to independently scale the size of the
        font elements.
    color_codes : bool
        If ``True`` and ``palette`` is a seaborn palette, remap the shorthand
        color codes (e.g. "b", "g", "r", etc.) to the colors from this palette.
    rc : dict or None
        Dictionary of rc parameter mappings to override the above.
    Examples
    --------
    .. include:: ../docstrings/set_theme.rst
    """
    set_context(context, font_scale)
    set_style(style, rc={"font.family": font})
    set_palette(palette, color_codes=color_codes)
    if rc is not None:
        mpl.rcParams.update(rc)


In [ ]:
#sns.set_theme(style="whitegrid", palette="muted")



# Draw a categorical scatterplot to show each observation
#ax = sns.swarmplot(data=co2_index, x="Country_Name", y="CO2_tonnes", hue="Region_x")
#ax.set(ylabel="")

In [ ]:
#Numerical data distribution

co2_num = co2_index.select_dtypes(include = ['float64', 'int64','datetime64'])
co2_num.info()

In [ ]:
#print(co2_index['Year'].describe())

In [ ]:
print(co2_num.describe().transpose())

In [ ]:
#Plot histograms of numeric data

co2_index.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
#too large
#sns.pairplot(co2_index)

In [ ]:
#plt.figure(figsize=(20,20))
#sns.heatmap(corr,annot=True)

In [ ]:
# Separate features and labels
X, y = co2_index[['World Rank','Property Rights', 'Judical Effectiveness','Government Integrity','Tax Burden','Business Freedom','Trade Freedom', 'Financial Freedom', 'GDP Growth Rate (%)']].values, co2_index['CO2_tonnes'].values
print('Features:',X[:10], '\nLabels:', y[:10], sep='\n')

In [ ]:
from sklearn.model_selection import train_test_split

# Split data 70%-30% into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

print ('Training Set: %d rows\nTest Set: %d rows' % (X_train.shape[0], X_test.shape[0]))

In [ ]:
# Train the model
from sklearn.linear_model import LinearRegression

# Fit a linear regression model on the training set
model = LinearRegression().fit(X_train, y_train)
#print (model)

In [ ]:
predictions = model.predict(X_test)
np.set_printoptions(suppress=True)
print('Predicted labels: ', np.round(predictions)[:10])
print('Actual labels   : ' ,y_test[:10])

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Regression')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)

rmse = np.sqrt(mse)
print("RMSE:", rmse)

r2 = r2_score(y_test, predictions)
print("R2:", r2)

In [ ]:
from sklearn.linear_model import Lasso

# Fit a lasso model on the training set
model = Lasso().fit(X_train, y_train)
print (model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Regression')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_text

# Train the model
model = DecisionTreeRegressor().fit(X_train, y_train)
print (model, "\n")

# Visualize the model tree
tree = export_text(model)
print(tree)

In [ ]:
# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train the model
model = RandomForestRegressor().fit(X_train, y_train)
print (model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
# Train the model
from sklearn.ensemble import GradientBoostingRegressor

# Fit a lasso model on the training set
model = GradientBoostingRegressor().fit(X_train, y_train)
print (model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, r2_score

# Use a Gradient Boosting algorithm
alg = GradientBoostingRegressor()

# Try these hyperparameter values
params = {
 'learning_rate': [0.1, 0.5, 1.0],
 'n_estimators' : [50, 100, 150]
 }

# Find the best hyperparameter combination to optimize the R2 metric
score = make_scorer(r2_score)
gridsearch = GridSearchCV(alg, params, scoring=score, cv=3, return_train_score=True)
gridsearch.fit(X_train, y_train)
print("Best parameter combination:", gridsearch.best_params_, "\n")

# Get the best model
model=gridsearch.best_estimator_
print(model, "\n")

# Evaluate the model using the test data
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)
r2 = r2_score(y_test, predictions)
print("R2:", r2)

# Plot predicted vs actual
plt.scatter(y_test, predictions)
plt.xlabel('Actual Labels')
plt.ylabel('Predicted Labels')
plt.title('Predictions')
# overlay the regression line
z = np.polyfit(y_test, predictions, 1)
p = np.poly1d(z)
plt.plot(y_test,p(y_test), color='magenta')
plt.show()

In [ ]:
world = pd.read_csv("/kaggle/input/countries-of-the-world/countries of the world.csv")

In [ ]:
world.info()

In [ ]:
# counting unique values 
n = world.nunique(axis=0) 
  
print("No.of.unique values in each column :\n", 
      n)

In [ ]:
df.drop_duplicates(subset=['brand', 'style'], keep='last')i